In [1]:
%cd /content/drive/MyDrive/colab/distillog/ablation_study/Spirit5M-kd-chronological

/content/drive/MyDrive/colab/distillog/ablation_study/Spirit5M-kd-chronological


In [2]:
!pip install torchinfo
!pip install overrides

In [6]:
!python train.py

Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/usr/local/lib/python3.10/dist-packages/torch/_prims_common/__init__.py", line 23, in <module>
    import sympy
  File "/usr/local/lib/python3.10/dist-packages/sympy/__init__.py", line 74, in <module>
    from .polys import (Poly, PurePoly, poly_from_expr, parallel_poly_from_expr,
  File "/usr/local/lib/python3.10/dist-packages/sympy/polys/__init__.py", line 123, in <module>
    from .partfrac import apart, apart_list, assemble_partfrac_list
  File "/usr/local/lib/python3.10/dist-packages/sympy/polys/partfrac.py", line 15, in <module>
    def apart(f, x=None, full=False, **options):
  File "/usr/local/lib/python3.10/dist-packages/sympy/utilities/decorator.py", line 76, in xthreaded
    return threaded_factory(func, False)
  Fi

In [ ]:
!python teach.py

In [10]:
!python test.py

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
[0.95357186 0.9646551  0.9556816  0.9641209  0.95396215 0.9535593
 0.96378905 0.9601419  0.9509677  0.95286876 0.9596711  0.95750135
 0.9639208  0.95201296 0.9668633  0.95422    0.9578903  0.9618708
 0.95550853 0.9662376  0.9547215  0.95585823 0.95907885 0.955622
 0.96222496 0.95968187 0.9571585  0.9028274  0.9618311  0.89167595
 0.82172585 0.88632125 0.8674021  0.85287136 0.8604925  0.8742274
 0.8793388  0.8604925  0.85109824 0.8722843  0.85504943 0.92141235
 0.9615106  0.9705508  0.9708162  0.9721644  0.9732791  0.9732791
 0.9732039  0.970768  ] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0.96299773 0.95746714 0.9636435  0.9538

In [ ]:
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import Parameter
from torch.nn.modules.module import Module
from tqdm import tqdm
import math
import csv
from time import time
from torchinfo import summary
from utils import save_model, train, read_data, load_data
from utils import DistilLog
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = 2
batch_size = 50
learning_rate = 0.01
hidden_size = 128
input_size = 30
sequence_length = 50
num_layers = 2

train_path = '../../datasets/Spirit5M/chronological_train.csv'
save_teacher_path = '../../datasets/Spirit5M/model/chronological_teacher.pth'
save_noKD_path = '../../datasets/Spirit5M/model/chronological_noKD.pth'

Teacher = DistilLog(input_size, hidden_size, num_layers, num_classes, is_bidirectional=False).to(device)


#noKD = train(noKD, train_loader, learning_rate, num_epochs = 40)
#save_model(noKD, save_noKD_path)

In [29]:
inp = torch.rand((1, 50, 30))
out, _ = Teacher(inp)
print(inp, out)

inp2 = torch.rand((1, 50, 30))
out2, _ = Teacher(inp)
print(inp2, out2)

tensor([[[0.4305, 0.8887, 0.3183,  ..., 0.9179, 0.6782, 0.2068],
         [0.7448, 0.9167, 0.2724,  ..., 0.9651, 0.3890, 0.0807],
         [0.4900, 0.1132, 0.8900,  ..., 0.7621, 0.1479, 0.4506],
         ...,
         [0.7560, 0.4241, 0.2032,  ..., 0.7236, 0.9254, 0.9178],
         [0.7848, 0.2864, 0.2564,  ..., 0.1365, 0.4084, 0.7594],
         [0.7575, 0.5877, 0.1441,  ..., 0.8781, 0.0096, 0.5897]]]) tensor([[ 2.4744, -2.6180]], grad_fn=<AddmmBackward0>)
tensor([[[0.8124, 0.4506, 0.4334,  ..., 0.0161, 0.7689, 0.3848],
         [0.7213, 0.2515, 0.3290,  ..., 0.9053, 0.8897, 0.2215],
         [0.8733, 0.9216, 0.6368,  ..., 0.9754, 0.8945, 0.1004],
         ...,
         [0.5861, 0.6004, 0.3696,  ..., 0.3223, 0.1997, 0.2432],
         [0.5240, 0.3752, 0.6925,  ..., 0.7506, 0.2597, 0.1681],
         [0.5913, 0.6715, 0.3240,  ..., 0.0156, 0.4606, 0.0766]]]) tensor([[ 2.4701, -2.6123]], grad_fn=<AddmmBackward0>)


In [28]:
from utils import load_data, load_model, DistilLog
Teacher = load_model(Teacher, save_teacher_path)

In [ ]:
for name, param in Teacher.named_parameters():
  print(f"Parameter name: {name}")
  print(f"Parameter shape: {param.shape}")
  print(f"Parameter values: {param}")
  print()
